In [4]:
import pydicom

In [1]:


folder = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/dicom/DEV308_20220906_133720/Series_13_ROC1/'

#get dicoms in the relevant folder
import os
dicoms = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.dcm')]




In [6]:
# Load the first DICOM file
dcm = pydicom.dcmread(dicoms[0])
# Print the file's meta-information
print(dcm)

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 212
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.19.45348.2022090614104429742965810
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.2
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_362'
(0002, 0016) Source Application Entity Title     AE: 'AWP45348'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'MB', 'ND', 'NORM', 'MOSAIC']
(0008, 0012) Instance Creation Date              DA: '20220906'
(0008, 0013) Instance Creation Time              TM: '141044.8110

Running stuff Jolinda suggested

In [7]:
import nibabel as nib

In [11]:


img_filepath = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/sub-DEV309/ses-wave2/func/sub-DEV309_ses-wave2_task-ROC_acq-1_bold.nii.gz'

img = nib.load(img_filepath)

newimg = nib.Nifti1Image(img.get_fdata(), img.header.get_base_affine())

new_filepath = img_filepath.replace('.nii.gz', '_base_affine.nii.gz')
#add a "_base_affine suffix to the filepath before the extension"
newimg.to_filename(new_filepath)

In [22]:
import pandas as pd
import nibabel as nib
import shutil


#read in the csv file, setting a custom header since there is none in the csv
df_runs = pd.read_csv('runs_to_reset.csv', header=None, names=['subid', 'runid'])



In [ ]:
bids_path = '/gpfs/projects/sanlab/shared/DEV/bids_data/'

In [23]:
bids_path = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/'

In [24]:
for row in df_runs.itertuples():
    #get the subject id and run id
    subid = row.subid
    runid = row.runid
    
    #get the filepath to the nifti file
    img_folder = bids_path + f'sub-{subid}/ses-{runid}/func/'

    #get a list of the nifti files in the folder
    if os.path.exists(img_folder):
        img_files = [f for f in os.listdir(img_folder) if f.endswith('.nii.gz')]
    else:
        print(f'No folder found for sub-{subid} run-{runid}')
        continue
    #print the number of images found.
    print(f'Found {len(img_files)} images for sub-{subid} run-{runid}')


    #make a backup folder
    backup_folder = img_folder.replace('bids_data/',f'bids_data/derivatives/affine_correction_originals/')
    run_backup_folder = backup_folder + f'sub-{subid}/ses-{runid}/func/'
    if not os.path.exists(run_backup_folder):
        os.makedirs(run_backup_folder)

    #loop through the nifti files
    for img_file in img_files:
        #get the full filepath
        img_filepath = os.path.join(img_folder, img_file)
        #make a backup copy of the image
        shutil.copy(img_filepath, img_filepath.replace(bids_path, backup_folder))
        #load the nifti file
        img = nib.load(img_filepath)
        #get the affine matrix
        affine = img.header.get_base_affine()
        #reset the affine matrix
        newimg = nib.Nifti1Image(img.get_fdata(), affine)
        #save the file
        newimg.to_filename(img_filepath)

No folder found for sub-DEV034 run-wave1
No folder found for sub-DEV041 run-wave1
No folder found for sub-DEV084 run-wave2
No folder found for sub-DEV123 run-wave1
No folder found for sub-DEV194 run-wave2
Found 14 images for sub-DEV309 run-wave2
